# Tarea 3 Métodos No Lineales
---
## Integrantes: 
- *José Eduardo Caimapo* -  jose.caimapo.12@sansano.usm.cl - 201373573-8
- *Rodrigo Sepúlveda* - rodrigo.sepulveda.12@sansano.usm.cl - 201204766-8

In [ ]:
import matplotlib.pyplot as plt
import numpy             as np
from   sklearn.utils           import check_random_state
from   sklearn.model_selection import train_test_split

## 1. Small Circle inside Large Circle

El objetivo de esta sección es experimentar con algunos modelos no-lineales sobre un problema de juguete generado  para visualizar  algoritmos  de clustering.  Se  trata  de  un  problema  de  clasificación  a  todas  luces linealmente inseparable, en el sentido que, si denotamos por $x \in I\!R^{2}$ un patrón de entrada y por $y \in \{1, -1\}$ su correspondiente etiqueta, no existen $w \in I\!R^2,b \in I\!R$ tal que $y(w^T x + b) \geq p \gt 0$
El problema nos permite 
hacer un recorrido rápido por las grandes ideas en la búsqueda de la ***no-linealidad***.



#### a) Escriba una función que genere n datos etiquetados de la forma $\{(x_1,y_1),...,(x_n,y_n)\}$ con $x_i \in I\!R^{2}, y_i \in \{ 0 , 1\}$, con una distribución de probabilidad que refleje la configuración linealmente inseparable que muestra la Fig 1. Utilice esta función para crear 1000 datos de entrenamiento y 1000 datos de pruebas. Para medir la tendencia de los modelos a ***sobre ajuste***, agregue un $5 \%$ de ruido al dataset, generando x's cercanos a la frontera. Genere un gráfico que muestre datos de entrenamiento y pruebas, identificando cada clase con un color diferente.

In [3]:
def do_circles(n=2000,noisy_n=0.05):
    generator = check_random_state(10)
    linspace = np.linspace(0,2*np.pi,n // 2+1)[:-1]
    outer_circ_x = np.cos(linspace)
    outer_circ_y = np.sen(linspace)
    inner_circ_x = outer_circ_x * .3
    inner_circ_y = outer_circ_y * .3
    X = np.vstack((np.append(outer_circ_x,inner_circ_x),
                   np.append(outer_circ_y,inner_circ_y))).T
    y = np.hstack([np.zeros(n , dtype=np.intp),
                   np.ones(n, dtype=np.intp)])
    X += generator.normal(scale=noisy_n, size=X.shape)
    
    X_train, X_test, y_train, y_test = train_test_split(X,y,
                                                        test_size=0.5,
                                                        random_state=42)
    return X_train,y_train,X_test,y_test
    

In [ ]:
def plot_classifier(clf,X_train,Y_train,X_test,Y_test,model_type):
    f, axis = plt.subplots(1, 1, sharex='col', sharey='row',figsize=(12, 8))
    axis.scatter(X_train[:,0],X_train[:,1],s=30,c=Y_train,zorder=10,cmap='cool')
    axis.scatter(X_test[:,0],X_test[:,1],s=20,c=Y_test,zorder=10,cmap='Greys')
    XX, YY = np.mgrid[-2:2:200j, -2:2:200j]
    if model_type == 'tree':
        Z = clf.predict_proba(np.c_[XX.ravel(), YY.ravel()])[:,0]
    elif model_type == 'ann':
        Z = clf.predict(np.c_[XX.ravel(), YY.ravel()])
    else: raise ValueError('model type not supported')
        Z = Z.reshape(XX.shape)
    Zplot = Z >= 0.5
    axis.pcolormesh(XX, YY, Zplot ,cmap='YlGn')
    axis.contour(XX, YY, Z, alpha=1, colors=["k", "k", "k"], linestyles=["--", "-", "--"],
    levels=[-2, 0, 2])
    plt.show()

#### b) Demuestre  experimentalmente que  una  red  neuronal  artificial  correspondiente  a  1  sola  neurona  (i.e.sin capas escondidas) no puede resolver satisfactoriamente el problema. Puede utilizar la funci ́on de activaci ́on y el m ́etodo deentrenamiento que prefiera. Sea convincente: por ejemplo, intente modificar los par ́ametros de la m ́aquina de aprendizaje,reportando m ́etricas que permitan evaluar el desempeño del modelo en el problema con cada cambio efectuado.Adapte tambi ́en la funci ́on plot classifier para que  represente  gr ́aficamente  la  soluci ́on  encontrada  por  la  red  neuronal. Describa  y  explique  lo  que observa, reportando gr ́aficos de la soluci ́on s ́olo para algunos casos representativos.

#### c)  Demuestre experimentalmente que una red neuronal artificial con 1 capa escondida puede resolver satisfactoriamente el problema obtenido en (a). Puede utilizar la arquitectura y el método de entrenamiento que prefiera, pero en esta actividad puede optar tranquilamente por usar los hiper-parámetros que se 3 entregan como referencia en el código de ejemplo. Cambie el número de neuronas N h en la red entre 2 y 32 en potencias de 2, graficando el error de entrenamiento y pruebas como  función de N h . Describa y explique lo que observa. Utilice la función plot classifier , diseñada anteriormente, para construir gráficos de la solución en algunos casos representativos

#### d) Demuestre experimentalmente que stump (árbol de clasificación de 1 nivel) no puede resolver satisfactoriamente el problema anterior. Puede utilizar el criterio y la función de partición que prefiera. Sea convincente:  por  ejemplo,  intente  modificar  los  parámetros  de  la  máquina,  reportando  métricas  que permitan evaluar el desempeño del modelo en el problema con cada cambio efectuado. Adapte también la función plot classifier para que represente gráficamente la solución encontrada por el árbol. Describa y explique lo que observa, reportando gráficos de la solución sólo para algunos casos representativos.

#### e) Demuestre experimentalmente que un árbol de clasificación de múltiples niveles puede resolver satis- factoriamente el problema estudiado. Puede utilizar el criterio y la función de partición que prefiera,pero puede optar tranquilamente por usar los hiper-parámetros que se entregan como referencia en el código de ejemplo. Cambie el número de niveles admitidos en el árbol N t entre 2 y 20, graficando el error de entrenamiento y pruebas como función de N t. Describa y explique lo que observa. Utilice la función plot classifier, diseñada anteriormente, para construir gráficos de la solución en algunos casos representativos.


#### f) Como ya se demostró experimentalmente que este problema es linealmente inseperable, ahora se pideexperimentar otra alternativa. Para ello deberá realizar una proyección de los datos a un nuevo espacio dimensional (manifold) en el cual se reconozcan sus patrones no lineales, para poder trabajarlos confronteras lineales. Utilice la técnica de PCA con la ayuda de un Kernel  Gaussiano para extraer sus vectores con dimensión infinita de mayor varianza.

#### g) Ajuste un algoritmo de aprendizaje con fronteras lineal para los datos proyectados en este nuevo espacio que captura sus  componentes no lineales, muestre gráficamente que el problema ahora puede ser resulto con estos métodos. Reporte métricas para evaluar el desempeño, comente y concluya.